In [1]:
import sys, os, time, h5py, sys, pickle, matplotlib  
sys.path.insert(0, "/home/ruthvik/Desktop/Summer 2017/SpikeFlow")
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  ### GPU is not used
import numpy as np
import spikeflow as sf
from spikeflow import network as network
import matplotlib.animation as animation
from spikeflow import inputlayerclass as inputlayer
np.random.seed(0)

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters




Prepping the MNIST data .... 

Finished MNIST data ..

Prepping the EMNIST data .... 

Finished EMNIST data ..

Prepping the CIFAR100 data .... 

Unable to find data in AllDataSets/cifar100..
Prepping the CIFAR10 data .... 

Unable to find data in AllDataSets/cifar10..
